In [4]:
import requests
import pandas as pd

# Function to fetch data from the API
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors
        return response.json()  # Return JSON data
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except ValueError as e:
        print(f"JSON decode error: {e}")
        return []

# API endpoints
travels_url = 'http://localhost:8000/list/'
checklists_url = 'http://localhost:8000/checklist/list/'

# Fetch data from the endpoints
travels_data = fetch_data(travels_url)
checklists_data = fetch_data(checklists_url)

# Convert to Pandas DataFrames
travels_df = pd.DataFrame(travels_data)
checklists_df = pd.DataFrame(checklists_data)

# Forward fill null values
travels_df = travels_df.ffill().bfill()  # Fill missing values forward and backward
checklists_df = checklists_df.ffill().bfill()

# Print DataFrames
print("Travels DataFrame:")
print(travels_df.head())

print("\nChecklists DataFrame:")
print(checklists_df.head())

# Merge travels and checklists if they share a relationship (assuming 'id' and 'checklists' are related)
merged_df = pd.merge(
    travels_df,
    checklists_df,
    left_on='id',  # Adjust based on your models
    right_on='id',  # Adjust based on your models
    how='left'
)

# Print merged DataFrame
print("\nMerged DataFrame:")
print(merged_df.head())


Travels DataFrame:
   id            name                                            country  \
0   2   Shelby Daniel  Consectetur qui laboriosam dolore fugiat quaer...   
1   3   Jasper Howard  Quaerat sed cupidatat ipsum qui et repellendus...   
2   4  Abra Macdonald  Est repellendus Excepteur iure architecto eu p...   
3   5     Winter Bush  Eaque quas veniam ea deleniti non in dolores v...   
4   7     Lana Conner  Sequi iure quis tempor sunt recusandae Nulla a...   

                                                city  \
0  Voluptatem quo aut dolor dolorem excepturi mag...   
1  Necessitatibus eligendi minim quibusdam aut ut...   
2  Adipisci rerum inventore similique voluptatum ...   
3  Illum duis corporis magna sit aut aute in blan...   
4                 Quasi at sint ullamco perspiciatis   

                                         description  user_id  
0  Fugit beatae voluptate cupidatat unde sed cum ...        2  
1  Ut id odio cupiditate illo voluptate deleniti ...       

In [3]:
import requests
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Function to fetch data from the API

def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  
        return response.json() 
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except ValueError as e:
        print(f"JSON decode error: {e}")
        return []

# API endpoints
travels_url = 'http://localhost:8000/list/'
checklists_url = 'http://localhost:8000/checklist/list/'

# Fetch data from the endpoints

travels_data = fetch_data(travels_url)
checklists_data = fetch_data(checklists_url)

# Convert to Pandas DataFrames

travels_df = pd.DataFrame(travels_data)
checklists_df = pd.DataFrame(checklists_data)

# Handle missing values by forward and backward filling

travels_df = travels_df.ffill().bfill()
checklists_df = checklists_df.ffill().bfill()

# Data Cleaning: Removing duplicates

travels_df = travels_df.drop_duplicates()
checklists_df = checklists_df.drop_duplicates()

# Feature Engineering: Add derived features
travels_df['name_length'] = travels_df['name'].apply(len)
checklists_df['description_length'] = checklists_df['description'].apply(len)

# Standardization: Scale numeric features using StandardScaler
scaler = StandardScaler()
numeric_features = ['name_length']  # Add other numeric columns as needed
travels_df[numeric_features] = scaler.fit_transform(travels_df[numeric_features])

# Normalization: Scale numeric features to a range [0, 1] using MinMaxScaler
normalizer = MinMaxScaler()
checklist_numeric_features = ['description_length']  # Add other numeric columns as needed
checklists_df[checklist_numeric_features] = normalizer.fit_transform(checklists_df[checklist_numeric_features])

# Print DataFrames
print("Travels DataFrame:")
print(travels_df.head())

print("\nChecklists DataFrame:")
print(checklists_df.head())

# Export individual DataFrames to CSV files
output_dir = "output_csvs"
os.makedirs(output_dir, exist_ok=True) 

travels_csv_path = os.path.join(output_dir, "travels.csv")
checklists_csv_path = os.path.join(output_dir, "checklists.csv")

travels_df.to_csv(travels_csv_path, index=False)
checklists_df.to_csv(checklists_csv_path, index=False)

print(f"\nTravels DataFrame exported to {travels_csv_path}")
print(f"Checklists DataFrame exported to {checklists_csv_path}")

# Merge travels and checklists on 'id' (assuming they share a relationship)
merged_df = pd.merge(
    travels_df,
    checklists_df,
    left_on='id',  
    right_on='id',
    how='left',
    suffixes=('_travel', '_checklist')
)

# Scaling and Normalizing merged data
numeric_columns = ['name_length', 'description_length']  
merged_df[numeric_columns] = scaler.fit_transform(merged_df[numeric_columns])

# Print and export the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df.head())

merged_csv_path = os.path.join(output_dir, "merged_data.csv")
merged_df.to_csv(merged_csv_path, index=False)

print(f"\nMerged DataFrame exported to {merged_csv_path}")


Travels DataFrame:
   id            name                                            country  \
0   2   Shelby Daniel  Consectetur qui laboriosam dolore fugiat quaer...   
1   3   Jasper Howard  Quaerat sed cupidatat ipsum qui et repellendus...   
2   4  Abra Macdonald  Est repellendus Excepteur iure architecto eu p...   
3   5     Winter Bush  Eaque quas veniam ea deleniti non in dolores v...   
4   7     Lana Conner  Sequi iure quis tempor sunt recusandae Nulla a...   

                                                city  \
0  Voluptatem quo aut dolor dolorem excepturi mag...   
1  Necessitatibus eligendi minim quibusdam aut ut...   
2  Adipisci rerum inventore similique voluptatum ...   
3  Illum duis corporis magna sit aut aute in blan...   
4                 Quasi at sint ullamco perspiciatis   

                                         description  user_id  name_length  
0  Fugit beatae voluptate cupidatat unde sed cum ...        2     0.500000  
1  Ut id odio cupiditate illo vol